In [ ]:
import sys
from copy import deepcopy
import time

from PIL import Image, ImageDraw

sys.path.append('../utils')
from pyutils import *

In [ ]:
puzzle = readutf8('input.txt')
sample = readutf8('sample.txt')

In [ ]:
maze = strtomat(puzzle)
start, end = Pt(len(maze) - 2, 1), Pt(1, len(maze[0]) - 2)

In [ ]:
# from PIL import Image, ImageDraw, ImageColor
# def heatmap(mat: StrMatrix, distances: dict[Pt, int], dmin: int, dmax: int,
#         highlight: list[Pt] | None = None,
#         out: tuple[int, int]=(-1, -1)
#     ) -> Image.Image:
#     highlight = highlight or []
#     im = Image.new(size=(len(mat[0]), len(mat)), mode='RGB', color='black')
#     brush = ImageDraw.Draw(im)
#     for pt, val in mat_iter(mat):
#         if val != '#':
#             brush.point((pt[1], pt[0]), 'white')
#         if pt in distances:
#             perc = (distances[pt] - dmin) / (dmax - dmin)
#             hue = int(120 * perc)
#             brush.point((pt[1], pt[0]), f'hsv({hue}, 100%, 100%)')
#     for pt in highlight:
#         brush.point((pt[1], pt[0]), 'blue')
#     if all(n > 0 for n in out):
#         im = im.resize(out, resample=0)
#         brush = ImageDraw.Draw(im)
#     gridw = len(mat[0])
#     gridh = len(mat)
#     for pt, d in distances.items():
#         x = pt[1] * (im.size[0] / gridw)
#         y = pt[0] * (im.size[1] / gridh)
#         # print(x, y, str(d))
#         brush.text((x, y), str(d), fill='black')
#     return im

In [ ]:
# def walk_maze(mat: StrMatrix, start_char: str, end_char: str):
#     mat = deepcopy(mat)

#     start_pt: Pt = Pt(-1, -1)
#     end_pt: Pt = Pt(-1, -1)
#     for pt, val in mat_iter(mat):
#         if val == start_char:
#             start_pt = Pt.of(pt)
#         if val == end_char:
#             end_pt = Pt.of(pt)
#         if Pt(-1, -1) not in [start_pt, end_pt]:
#             break

#     assert(start_pt != Pt(-1, -1))
#     assert(end_pt != Pt(-1, -1))

#     visited: dict[Pt, int] = {}
#     vertices: list[Pt] = [start_pt]
#     segments: dict[Pt, list[Pt]] = {start_pt: []}
#     pos = start_pt
#     dirpt: Pt = Pt(0, 1)
#     score: int = 0

#     log: list[str] = ['...']
#     def _next_branch(pos: Pt, vertex: Pt) -> bool | None:
#         """
#         Returns True if end was reached, False if wall was reached, None otherwise
#         """
#         nonlocal dirpt, score
    
#         # time.sleep(0.1)
#         # clear_output(wait=True)
#         # display(heatmap(mat, visited, 0, end_pt.distance(start_pt), highlight=vertices))
#         # print(log[-1])
#         # print(f'Now: {pos} ; {dirpt}')
#         # print(f'Score: {score}')
#         # display(heatmap(mat, visited, 0, end_pt.distance(start_pt)).resize((400, 400), resample=0))
#         # print(show_in_matrix(mat, *visited, col='white on blue', colmap={pos:'black on red'}))

#         visited[pos] = pos.distance(start_pt)

#         branches: list[Pt] = []
#         for branch in Pt.cardinals():
#             future = pos + branch
#             if future in visited:
#                 continue
#             val = matget(mat, future)
#             if val == '#':
#                 continue
#             branches.append(branch)

#         if len(branches) > 1:
#             vertices.append(pos)
#             segments[pos] = []
#             vertex = pos
            
#         pos_is_vertex = pos == vertex

#         for branch in branches:
#             future = pos + branch
#             if pos_is_vertex:
#                 segments[vertex].clear()
#             segments[vertex].append(future)
#             log.append(f'Checking branch: {branch} off of vertex {vertex}')
#             if future == end_pt:
#                 visited[future] = future.distance(start_pt)
#                 log.append(f'{future}: is END')
#                 # return True
#             if future in visited:
#                 log.append(f'{future}: visited')
#                 continue
#             if matget(mat, pos + branch) == '#':
#                 log.append(f'{future}: is a wall')
#                 continue
#             if dirpt != branch:
#                 diff = abs(Pt.cardinals().index(branch) - Pt.cardinals().index(dirpt))
#                 if diff == 2:
#                     score += 2000
#                 if diff in [1, 3]:
#                     score += 1000
#             dirpt = branch
#             if _next_branch(future, vertex) is True:
#                 return True
#         segments[vertex].clear()
#         if pos_is_vertex:
#             log.append(f'{vertex} is out of branches')
#             del segments[vertex]
#         else:
#             log.append('Out of branches')

#     ret = _next_branch(pos, pos)
#     successful: dict[Pt, int] = {}
#     for k, v in segments.items():
#         if k not in successful:
#             successful[k] = visited[k]
#         for pt in v:
#             if pt not in successful:
#                 successful[pt] = visited[pt]
#     display(heatmap(mat, visited, 0, end_pt.distance(start_pt), highlight=vertices, out=(600,600)))
#     # display(heatmap(mat, {pt:d for pt, d in successful.items()}, 0, end_pt.distance(start_pt), highlight=vertices).resize((600, 600), resample=0))
#     return ret, visited, log, successful

In [ ]:
# result = walk_maze(maze, 'S', 'E')

In [ ]:
def path(mat: Matrix, start: Pt, end: Pt):
    visited: set[Pt] = set()
    vertices: dict[Pt, dict[Pt, list[Pt]]] = {}

    def _check_surrounding(pt: Pt, anchor: Pt) -> list[Pt]:
        branches: list[Pt] = []
        for direc in Pt.cardinals():
            future = pt + direc
            if future in visited:
                if not (future in unchecked_verts):
                    continue
            if matget(mat, future) == '#':
                continue
            branches.append(future)
        return branches
    
    def _explore(cursor: Pt, line_acc: list[Pt], anchor: Pt) -> tuple[bool, list[Pt]] | None:
        while True:
            visited.add(cursor)
            if cursor == end:
                return True, line_acc
            branches: list[Pt] = _check_surrounding(cursor, anchor)
            if len(branches) == 0:
                return False, line_acc
            if len(branches) == 1:
                cursor = branches[0]
                line_acc.append(cursor)
                if cursor in unchecked_verts:
                    return
                continue
            if len(branches) > 1:
                unchecked_verts.add(cursor)
                vertices[cursor] = {i:[i] for i in _check_surrounding(cursor, anchor)}
                return
    
    cursor: Pt = start
    vertices[cursor] = {i:[i] for i in _check_surrounding(cursor, cursor)}
    checked_verts: list[Pt] = []
    unchecked_verts: set[Pt] = set()
    shortest: list[Pt] = [start]
    dead: list[Pt] = []
    while True:
        # time.sleep(0.25)
        # clear_output(wait=True)
        # display(matimg(mat, {
        #     (lambda pt, v: v == '#'): 'black',
        #     (lambda pt, v: pt in visited): 'red',
        #     (lambda pt, v: pt in shortest): 'blue'
        #     }).resize((100, 100), resample=0))
        
        visited.add(cursor)
        ret = None
        for branch, line in vertices[cursor].items():
            ret = _explore(branch, line, cursor)
            if ret is not None:
                if ret[0] == False:
                    dead.append(branch)
                if ret[0] == True:
                    shortest.extend(ret[1])
                    break
        checked_verts.append(cursor)
        if (ret is not None) and (ret[0] == True):
            break
        line_lengths: dict[int, Pt] = {len(v):k for k, v in vertices[cursor].items() if k not in shortest + dead}
        if not line_lengths:
            
            print(cursor)
            print(checked_verts)
            # print(vertices.keys())
            # print()
            # print(dead)
            # display(matimg(mat, {
            #     (lambda pt, v: v == '#'): 'black',
            #     (lambda pt, v: pt in visited): 'red',
            #     (lambda pt, v: pt in shortest): 'blue',
            #     (lambda pt, v: pt == cursor): 'lime'
            # }).resize((1000,1000), resample=0))
        min_line = line_lengths[min(line_lengths.keys())]
        shortest.extend(vertices[cursor][min_line])
        unchecked_verts = {i for i in unchecked_verts if i == shortest[-1]}
        if len(unchecked_verts) > 0:
            cursor = unchecked_verts.pop()
            continue
        else:
            break
    
    display(matimg(mat, {
        (lambda pt, v: v == '#'): 'black',
        (lambda pt, v: pt in visited): 'red',
        (lambda pt, v: pt in shortest): 'blue'
    }).resize((400,400), resample=0))

    return shortest

In [ ]:
def score_path(path: list[Pt]) -> int:
    startdir = Pt(0, 1)
    score = 0
    if startdir != path[1] - path[0]:
        score += 1000
    for n, _ in enumerate(result):
        if n + 2 >= len(result):
            break
        a = result[n]
        b = result[n + 1]
        c = result[n + 2]
        if a[0] != c[0] and a[1] != c[1]:
            score += 1000
    score += (len(result) - 1)
    return score

In [ ]:
result = path(maze, start, end)
# Need to solve dead ends, backtrack to previous vertex and erase recently added points to shortest path list!

In [ ]:
score_path(result)